# Copredication Detection with Argument Semantic Type Classifiers

We apply our classifiers that are trained for the identification of argument semantic types in predications to detect copredication. First, we parse the sentences to identify verb-direct object and adjective-noun pairs in the sentences. We classify each predication with the relative classifiers. We select the sentences in which the predications are classified as positive by different semantic type classifiers.

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import spacy
import pickle

In [2]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")
nlp = spacy.load("en_core_web_trf")

type_one = "Artifact"
type_two = "Event"

with open(f"classifiers/{type_one}_verb_clf.pickle", "rb") as classifier_file:
    clf1_verb = pickle.load(classifier_file)
with open(f"classifiers/{type_one}_adj_clf.pickle", "rb") as classifier_file:
    clf1_adj = pickle.load(classifier_file)
with open(f"classifiers/{type_two}_verb_clf.pickle", "rb") as classifier_file:
    clf2_verb = pickle.load(classifier_file)
with open(f"classifiers/{type_two}_adj_clf.pickle", "rb") as classifier_file:
    clf2_adj = pickle.load(classifier_file)

c:\Users\yavas\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:732: UserWarning: [W095] Model 'en_core_web_trf' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
def get_relation_vector(tokenizer, model, sentence, items):
    item1_vector = get_averaged_vector(tokenizer, model, sentence, items[0])
    item2_vector = get_averaged_vector(tokenizer, model, sentence, items[1])
    if item1_vector is not None and item2_vector is not None:
        return np.concatenate((item1_vector, item2_vector), axis=1)


def get_averaged_vector(tokenizer, model, sentence, word):
    inputs = tokenizer(sentence, truncation=True, return_tensors="pt")
    word_char = sentence.index(word)
    token_id = inputs.char_to_token(word_char)
    output = model(**inputs, output_hidden_states=True).hidden_states
    vectors = [
        output[layer_no][:, token_id, :].detach().numpy() for layer_no in range(8, 12)
    ]
    return np.average(vectors, axis=0)


def get_verbobj(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == "VERB":
            for child in token.children:
                if child.dep_ == "dobj":
                    return (token.text, child.text)


def get_adjnoun(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == "NOUN":
            for child in token.children:
                if child.dep_ == "amod":
                    return (token.text, child.text)


def get_predications(rvector_verb, rvector_adj):
    verb_pred1 = clf1_verb.predict(np.asarray(rvector_verb, "float64").reshape(1, -1))
    adj_pred1 = clf1_adj.predict(np.asarray(rvector_adj, "float64").reshape(1, -1))
    verb_pred2 = clf2_verb.predict(np.asarray(rvector_verb, "float64").reshape(1, -1))
    adj_pred2 = clf2_adj.predict(np.asarray(rvector_adj, "float64").reshape(1, -1))
    return verb_pred1, adj_pred1, verb_pred2, adj_pred2


def detect_copredication(sentence):
    items_verb = get_verbobj(sentence)
    items_adj = get_adjnoun(sentence)
    rvector_verb = get_relation_vector(tokenizer, model, sentence, items_verb)
    rvector_adj = get_relation_vector(tokenizer, model, sentence, items_adj)

    if rvector_adj is not None and rvector_verb is not None:
        verb_pred1, adj_pred1, verb_pred2, adj_pred2 = get_predications(
            rvector_verb, rvector_adj
        )
        if verb_pred1 == [1] and adj_pred2 == [1]:
            print(f"Copredication Detected. Type: {type_one}-{type_two}")
        elif verb_pred2 == [1] and adj_pred1 == [1]:
            print(f"Copredication Detected. Type: {type_two}-{type_one}")
        elif verb_pred1 == [1] and adj_pred1 == [1]:
            print(f"Copredication NOT Detected." f" Type: {type_one}-{type_one}")
        elif verb_pred2 == [1] and adj_pred2 == [1]:
            print(f"Copredication NOT Detected." f" Type: {type_two}-{type_two}")

In [11]:
sentence = "She attended a delicious breakfast."
detect_copredication(sentence)

Copredication Detected. Type: Event-Artifact


In [10]:
sentence = "I cooked a romantic lunch."
detect_copredication(sentence)

Copredication Detected. Type: Artifact-Event


In [9]:
sentence = "I ate a delicious breakfast."
detect_copredication(sentence)

Copredication NOT Detected. Type: Artifact-Artifact


In [12]:
sentence = "She ate a hot soup."
detect_copredication(sentence)

Copredication NOT Detected. Type: Artifact-Artifact


In [13]:
sentence = "We organized a formal event."
detect_copredication(sentence)

Copredication NOT Detected. Type: Event-Event


In [14]:
sentence = "We attended a formal dinner."
detect_copredication(sentence)

Copredication NOT Detected. Type: Event-Event
